# Reconocimiento de digitos manuscritos utilizando una red neuronal

In [ ]:
# Dataset de training 60.000 imagenes
# Dataset de test 10.000 imagenes
# Las imagenes tienen 28x28 pixeles en escala de grises.
# El target es un número que indica el dígito, de 0 a 9 inclusive.

# Las imagenes fueron generadas por 250 personas diferentes, 
# la mitad estudiantes de educación media, la otra mitad empleados de Census Bureau.

# http://yann.lecun.com/exdb/mnist/



## Obtenemos dataset

In [ ]:
# Retorna una matriz de nxm, cada fila representa una imagen, cada columna un pixel.
# Los pixels están "desenrollados" en un vector de 784 features (28x28)

import os
import struct
import numpy as np


def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
        
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)
        
    return images, labels

In [ ]:
X_train, y_train = load_mnist('data/mnist', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

X_test, y_test = load_mnist('data/mnist', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

## Análisis de datos

In [ ]:
# Muestro un ejemplo de digito para cada tag.

%matplotlib notebook

import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=5)
ax = ax.flatten()

for i in range(10):
    img = X_train[y_train == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
plt.tight_layout

In [ ]:
# Muestro para un digito diferentes forma de representarlo

%matplotlib notebook

fig, ax = plt.subplots(nrows=5,
                        ncols=5)

ax = ax.flatten()
for i in range(25):
    img = X_train[y_train == 7][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
plt.tight_layout()
plt.show()

## Construcción del modelo

In [ ]:
# Importamos una implementación de red neuronal.
from neuralnet import NeuralNetMLP

In [ ]:
# Creamos una red neuronal.
# 784 de input, 50 unidades ocultas (neuronas en la capa oculta), y 10 unidades de salida.

epochs = 10
minibatches = 50

# l2: parámetro de regularizacion L2.
# epochs: número de pasadas por el dataset de entrenamiento.
# eta: Learning rate.
# alpha: Parámetro para aprendizaje del momento.
# decrease_const: Para adaptar el aprendizaje, ir decrementando el learning rate.
# shuffle: En cada epoch reordena aleatoriamente el dataset de entrenamiento.
# minibatches: Parte el dataset en minibatches partes, el gradiente es computado 
#              para cada minibatch, esto acelera el aprendizaje.
nn = NeuralNetMLP(n_output=10,
                n_features=X_train.shape[1],
                n_hidden=50,
                l2=0.1,
                l1=0.0,
                epochs=epochs,
                eta=0.001,
                alpha=0.001,
                decrease_const=0.00001,
                shuffle=True,
                minibatches=minibatches,
                random_state=1)

### Entrenamiento

In [ ]:
# Entrenamos

#nn.fit(X_train, y_train, print_progress=True)
nn.fit(X_train, y_train, print_progress=True, X_test=X_test, y_test=y_test)


In [ ]:
# Graficamos el costo en cada paso.
# Tenemos (minibatches x epochs) pasos.

%matplotlib notebook

plt.plot(range(len(nn.cost_)), nn.cost_, color="blue")
plt.plot(range(len(nn.test_cost_)), nn.test_cost_, color="red")
#plt.ylim([0, 20000])
plt.ylabel('Cost')
plt.xlabel('Epochs * '+str(minibatches))
plt.tight_layout()
plt.show()

In [ ]:
# Dibujamos una grafica más suave.
# Promedia el costo de los minibatches en cada epoch.

%matplotlib notebook

batches = np.array_split(range(len(nn.cost_)), epochs)
cost_ary = np.array(nn.cost_)
cost_avgs = [np.mean(cost_ary[i]) for i in batches]

plt.plot(range(len(cost_avgs)), cost_avgs, color='red')
#plt.ylim([0, 2000])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.tight_layout()
plt.show()


### Validación

In [ ]:
# Calculamos precision con el dataset de entrenamiento.

y_train_pred = nn.predict(X_train)

acc = np.sum(y_train == y_train_pred, axis=0) / X_train.shape[0]

print('Training accuracy: %.2f%%' % (acc * 100))

In [ ]:
# Ahora calculamos precisión con el dataset de test.

y_test_pred = nn.predict(X_test)

acc = np.sum(y_test == y_test_pred, axis=0) / X_test.shape[0]

print('Test accuracy: %.2f%%' % (acc * 100))

In [ ]:
# Vemos los casos en los cuales nuestra red se equivocó.

%matplotlib notebook

miscl_img = X_test[y_test != y_test_pred][:25]
correct_lab = y_test[y_test != y_test_pred][:25]
miscl_lab= y_test_pred[y_test != y_test_pred][:25]

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(25):
    img = miscl_img[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title('%d) t: %d p: %d' % (i+1, correct_lab[i], miscl_lab[i]))

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()